In [1]:
import os
import uuid
from dotenv import load_dotenv
from docx import Document
from datetime import datetime

# LangChain + Azure OpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import AzureOpenAIEmbeddings

# Azure Cognitive Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration
)

In [2]:
load_dotenv(dotenv_path=".env")

ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX", "docx-index")
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))

embeddings_model = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

EMBED_DIM = 3072
VPROFILE = "myHnswProfile"

In [3]:
ENDPOINT

'https://aisearchpilotos.search.windows.net'

In [3]:
def create_index():
    index_client = SearchIndexClient(endpoint=ENDPOINT, credential=credential)

    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
        SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=EMBED_DIM,
            vector_search_profile_name=VPROFILE
        )
    ]

    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw-vector-config-1", kind="hnsw")
        ],
        profiles=[
            VectorSearchProfile(name=VPROFILE, algorithm_configuration_name="my-hnsw-vector-config-1")
        ]
    )

    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            content_fields=[SemanticField(field_name="content")]
        )
    )

    semantic_search = SemanticSearch(configurations=[semantic_config])

    index = SearchIndex(
        name=INDEX_NAME,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search
    )

    result = index_client.create_or_update_index(index)
    print(f"Índice {result.name} creado/actualizado")

create_index()


Índice avatar-index creado/actualizado


In [4]:
def read_docx(filepath):
    doc = Document(filepath)
    text = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    return "\n".join(text)

FILEPATH = "productos_bancoomeva.docx"
full_text = read_docx(FILEPATH)
print(f"Texto total extraído ({len(full_text.split())} palabras)")


Texto total extraído (29969 palabras)


In [6]:
full_text

'Crédito de vivienda tradicional\nDescripción general\nCon esta línea de crédito, Bancoomeva te brinda una opción de financiación para adquirir vivienda nueva, usada o construirla en lote propio.\nCaracterísticas\nEl monto máximo de financiación es de 500 millones. Si eres cliente asociado a Coomeva el monto es de $850 millones.\nModalidades de crédito: UVR y Vivienda en pesos (tasa fija).\nSe cobra seguro de vida como valor adicional a la cuota. Las tarifas por este concepto se encuentran publicadas en Tarifas Vigentes o en las oficinas.\nPosibilidad de realizar abonos extras para disminuir la cuota o el plazo.\nOpción de adquirir el seguro de desempleo para asalariados o incapacidad total temporal para independientes, que te protegerá ante posibles siniestros.\nRequisitos mínimos\nPara adquirir el\xa0Crédito de Vivienda Tradicional, debes cumplir con los siguientes requisitos:\nSi eres empleado:\nCarta laboral no mayor a 30 días que especifique cargo, antigüedad, remuneración y tipo 

In [5]:
def chunk_text(text):
    text_splitter = SemanticChunker(embeddings_model, breakpoint_threshold_type="percentile")
    chunks = text_splitter.split_text(text)
    return chunks

chunks = chunk_text(full_text)
print(f"{len(chunks)} chunks generados")

77 chunks generados


In [6]:
def prepare_documents(chunks):
    docs = []
    for chunk in chunks:
        embedding = embeddings_model.embed_query(chunk)
        doc_id = str(uuid.uuid4())
        docs.append({
            "id": doc_id,
            "content": chunk,
            "embedding": embedding,
        })
    return docs

docs = prepare_documents(chunks)

In [7]:
def upload_to_search(docs):
    search_client = SearchClient(endpoint=ENDPOINT, index_name=INDEX_NAME, credential=credential)
    batch_size = 50

    for i in range(0, len(docs), batch_size):
        batch = docs[i:i+batch_size]
        result = search_client.upload_documents(documents=batch)
        for r in result:
            print(f"Key: {r.key}, Success: {r.succeeded}")

    print("Proceso completado")

upload_to_search(docs)

Key: 5de7dcbc-9421-4e72-a581-7929d229ef5d, Success: True
Key: 45551f69-8eaa-442c-a67b-4e090af0f8ae, Success: True
Key: 6f42cc06-1b11-419e-8c8b-29f297b93246, Success: True
Key: dbc0b6af-fd18-4bdc-8b7b-5a5b92c76576, Success: True
Key: c1f0f95d-e6c4-4217-9f3d-12ba130253ca, Success: True
Key: e119a013-f17c-4005-9d99-bafce7c00174, Success: True
Key: d50afac3-96ba-4e14-8667-c49c01ecdd19, Success: True
Key: 85742f02-0d37-407d-950c-e75925a180e2, Success: True
Key: c6ddd4c3-5c04-4df6-a751-924e2083503c, Success: True
Key: ca333725-2026-4fa6-be36-e1e1272c6021, Success: True
Key: 2bf2a101-a24d-4ce0-9db7-7a358c9d43e4, Success: True
Key: 580a0396-ffe5-4436-a857-7f411e7d6d2e, Success: True
Key: bbe41526-b3fc-4cb4-8f65-c2fa797d1bce, Success: True
Key: b3bf334e-073d-487a-a061-d6a34a22392e, Success: True
Key: 8b328447-14d0-40db-9206-b1054c23b741, Success: True
Key: 76c05966-1ab1-4b28-bc6d-c9caf20efa41, Success: True
Key: ae2ec93a-c46d-4d1e-b606-9144e96f1f3f, Success: True
Key: 5107fca6-0aa1-4bea-ad65-27

In [8]:
def search_products(query):
    """
    Realiza búsqueda vectorial + semántica en Azure Cognitive Search.
    - query: texto de la consulta
    """
    # Generar embedding de la query
    query_embedding = embeddings_model.embed_query(query)
    k = 3  # número de resultados a devolver
    
    # Conectar al índice
    search_client = SearchClient(endpoint=ENDPOINT, index_name=INDEX_NAME, credential=credential)

    # Ejecutar búsqueda híbrida
    results = search_client.search(
        search_text=query,
        vector_queries=[{
            "kind": "vector",
            "vector": query_embedding,
            "fields": "embedding",
            "k": k
        }],
        query_type="semantic",
        semantic_configuration_name="my-semantic-config",
        top=k
    )

    # Recopilar resultados
    output = []
    for result in results:
        output.append({
            "content": result["content"],
            "score": result["@search.score"]
        })

    return output

In [9]:
search_products('cuales son los requisitos minimos para abrir una cuenta de ahorros super tasa')

[{'content': 'Este servicio se encuentra disponible de Lunes a Domingo las 24 horas, sin embargo, por proceso de actualización de saldos en horas de la noche puede ocurrir que temporalmente esté "fuera de servicio". Cuenta de Ahorro Súper Tasa\nDescripción general\nLa Cuenta de Ahorros Súper Tasa te permite obtener una rentabilidad similar a la de un CDT y la disponibilidad inmediata de tus recursos como la de una Cuenta de Ahorro. Características:\nApertura desde $0 pesos. Sin saldo mínimo en cuenta. Cuenta de manejo individual. Aplica para clientes Persona Natural. Sin asignación de Tarjeta Débito\xa0o Talonario de Ahorro. Transferencias entre cuentas Bancoomeva y otros bancos (vía ACH) a través de canales electrónicos. Marcarla como exenta del 4x1000 (GMF) de acuerdo a los topes definidos por la ley. Para más información sobre el Seguro de Depósitos FOGAFIN, haz clic\xa0AQUÍ. Restricciones:\nLa Cuenta de Ahorro Súper Tasa es una cuenta diseñada para el ahorro, no es una cuenta trans